In [1]:
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA, KernelPCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import numpy as np
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

/home/dongjoon/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from skimage import feature

def calc_hog_features(X, image_shape=(28, 28), pixels_per_cell=(8, 8)):
    fd_list = []
    for row in X:
        img = row.reshape(image_shape)
        fd = hog(img, orientations=8, pixels_per_cell=pixels_per_cell, cells_per_block=(1, 1))
        fd_list.append(fd)
    
    return np.array(fd_list)

## **Fetching MNIST dataset and applying HoG or PCA.**

In [ ]:
#Fetching MNIST dataset
mnist = fetch_mldata('MNIST original')
X, y = mnist["data"], mnist["target"]
pca = PCA(n_components=100, svd_solver='randomized',
          whiten=True).fit(X)

######### Comment out either of below two lines for PCA or HoG
#X = calc_hog_features(X, pixels_per_cell=(8, 8))
X = pca.transform(X)
######### Comment out either of above two lines for PCA or HoG

x_train, x_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
np.save('x_train', x_train)
np.save('x_test', x_test)
np.save('y_train', y_train)
np.save('y_test', y_test)



## Trying XGboost

In [ ]:
'''param_list = [("eta", 0.08), ("max_depth", 6), ("subsample", 0.8), ("colsample_bytree", 0.8), ("objective", "multi:softmax"), ("eval_metric", "merror"), ("alpha", 8), ("lambda", 2), ("num_class", 10)]
n_rounds = 600
early_stopping = 50
    
d_train = xgb.DMatrix(x_train, label=y_train)
d_val = xgb.DMatrix(x_valid, label=y_valid)
eval_list = [(d_train, "train"), (d_val, "validation")]
bst = xgb.train(param_list, d_train, n_rounds, evals=eval_list, early_stopping_rounds=early_stopping, verbose_eval=True)
'''

## **Trying different simple ML algorithms to see its performance**

In [ ]:
#Training using ML algorithms
#Instantiate simple machine learning models

knn_clf = KNeighborsClassifier(n_neighbors=3)
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)

log_clf = LogisticRegression(solver='lbfgs', multi_class='multinomial')
ran_grid = [{'n_estimators':[50,100,150,200], 'max_depth':[9,10,11,12]}]
ran_clf =  RandomForestClassifier(n_estimators = 200, max_depth=12)

'''svc_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=7, C=0.5, gamma=0.05))
    ])'''
svc_clf = SVC(C=5,gamma=0.05)
per_clf = Perceptron()
nb_clf = GaussianNB()
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=100,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)

#SVC training
svc_clf.fit(x_train, y_train)
predicted = svc_clf.predict(x_test)
print('SVC with degree=7 test accuracy : {}'.format(accuracy_score(y_test, predicted)))
print('=============================================================================')
'''
#Gaussian Naive Bayes
nb_clf.fit(x_train, y_train)
nb_predicted = nb_clf.predict(x_test)
print('Gaussian Naive Bayes test accuracy : {}'.format(accuracy_score(y_test, nb_predicted)))
print('=============================================================================')

#Bagging or trees
bag_clf.fit(x_train, y_train)
bag_pred = bag_clf.predict(x_test)
print('Bagging decision tree test accuracy : {}'.format(accuracy_score(y_test, bag_pred)))
print('=============================================================================')

#Adaboosting of trees
ada_clf.fit(x_train, y_train)
ada_pred = ada_clf.predict(x_test)
print('Adaboosted deceision tree test accuracy : {}'.format(accuracy_score(y_test, ada_pred)))
print('=============================================================================')
'''
#Random Forrest regression
ran_clf.fit(x_train, y_train)
ran_predicted = ran_clf.predict(x_test)
print('Random Forest test accuracy : {}'.format(accuracy_score(y_test, ran_predicted)))
print('=============================================================================')

#Perceptron
per_clf.fit(x_train, y_train)
per_predicted = per_clf.predict(x_test)
print('Perceptron test accuracy: {}'.format(accuracy_score(y_test, per_predicted)))
print('=============================================================================')

#Logistic regression
log_clf.fit(x_train, y_train)
log_predicted = log_clf.predict(x_test)
print('Logistic regression test accuracy : {}'.format(accuracy_score(y_test, log_predicted)))
print('=============================================================================')

#KNN
knn_clf.fit(x_train, y_train)
knn_predicted = knn_clf.predict(x_test)
print('KNN test accuracy : {}'.format(accuracy_score(y_test, knn_predicted)))
print('=============================================================================')

# Train using simple ensemble voting classifier. Doesn't work well.
voting_clf = VotingClassifier(
    estimators=[
        ('lr', log_clf), 
        #('pf', per_clf),
        #('nb', nb_clf),
        #('ada', ada_clf),
        #('bag', bag_clf),
        #('rf', ran_clf), 
        ('svc', svc_clf),
        ('knn', knn_clf)],
    voting='hard')

voting_clf.fit(x_train, y_train)

train_predicted = voting_clf.predict(x_train)
voting_predicted = voting_clf.predict(x_test)

print('=============================================================================')
print('Train accuracy for simple voting emsemble learning: {}'.format(accuracy_score(y_train, train_predicted)))
print('Validation accuracy for simple voting emsemble learning: {}'.format(accuracy_score(y_test, voting_predicted)))
print('=============================================================================')


In [ ]:



voting_clf = VotingClassifier(
    estimators=[
        ('lr', log_clf), 
        #('pf', per_clf),
        #('nb', nb_clf),
        #('ada', ada_clf),
        #('bag', bag_clf),
        #('rf', ran_clf), 
        ('svc', svc_clf),
        ('knn', knn_clf)],
    voting='hard')

voting_clf.fit(x_train, y_train)

train_predicted = voting_clf.predict(x_train)
voting_predicted = voting_clf.predict(x_test)

print('=============================================================================')
print('Train accuracy for simple voting emsemble learning: {}'.format(accuracy_score(y_train, train_predicted)))
print('Validation accuracy for simple voting emsemble learning: {}'.format(accuracy_score(y_test, voting_predicted)))
print('=============================================================================')

voting_clf = VotingClassifier(
    estimators=[
        ('lr', log_clf), 
        ('pf', per_clf),
        #('nb', nb_clf),
        #('ada', ada_clf),
        #('bag', bag_clf),
        ('knn', knn_clf),
        ('rf', ran_clf), 
        ('svc', svc_clf)],
    voting='hard')

voting_clf.fit(x_train, y_train)

train_predicted = voting_clf.predict(x_train)
voting_predicted = voting_clf.predict(x_test)

print('=============================================================================')
print('Train accuracy for simple voting emsemble learning: {}'.format(accuracy_score(y_train, train_predicted)))
print('Validation accuracy for simple voting emsemble learning: {}'.format(accuracy_score(y_test, voting_predicted)))
print('=============================================================================')